In [7]:
import psycopg2
from psycopg2 import sql
import random
import pandas as pd
from datetime import datetime

# Database connection parameters
conn = psycopg2.connect(
    dbname="internetOfThings",
    user="admin",
    password="admin",
    host="localhost",
    port="5432"
)


In [10]:
df = pd.read_csv('trainning.data', delimiter=';')
print(df.keys())

Index(['date', 'time', 'activity', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z'],
      dtype='object')


In [29]:
df_date = df['date']
df_time = df['time']
df_activity = df['activity']
print(df_time[0])
print(df_date[0])
print(df_activity[0])
print(datetime.now())

13:51:15:847724020
01/07/23
0
2024-11-28 15:30:59.077552


In [27]:
timestamps = []
# Loop through all rows in df_date and df_time
for date_str, time_str in zip(df_date, df_time):
    # Convert date to full year format
    date_parts = date_str.split("/")
    full_date = f"20{date_parts[2]}-{date_parts[1]}-{date_parts[0]}"
    
    # Convert time to required format (nanoseconds to microseconds)
    time_parts = time_str.split(":")
    time_str_with_microseconds = f"{time_parts[0]}:{time_parts[1]}:{time_parts[2]}.{str(int(time_parts[3]) // 1000).zfill(6)}"
    
    # Combine date and time into timestamp string
    timestamp = f"{full_date} {time_str_with_microseconds}"
    
    # Append the timestamp to the list
    timestamps.append(timestamp)

# Output the list of timestamps

In [30]:
weight = 85
height = 184
meters_per_second = 1.36

for timestamp_str, prediction in zip(timestamps, df_activity):
    try:
        # Save the original meters_per_second to restore after modification
        original_mps = meters_per_second
        
        # If prediction is 1, double the meters_per_second
        if prediction == 1:
            meters_per_second *= 2

        with conn.cursor() as cursor:
            # Insert data into the activity_table
            cursor.execute(
                """
                INSERT INTO activity_table (activity_time, activity_status, weight, height, meters_per_second)
                VALUES (%s, %s, %s, %s, %s)
                """,
                (timestamp_str, prediction, weight, height, meters_per_second)
            )
            conn.commit()
        
        # Restore meters_per_second for next iteration
        meters_per_second = original_mps
        
    except Exception as e:
        print(f"Cannot insert: {str(e)}")
